In [1]:
# Dependencies
import requests as req
import json
import tweepy
import pandas as pd
import time as t
import matplotlib.pyplot as plt
# from config import (consumer_key, consumer_secret, access_token, access_token_secret)
from datetime import datetime, date, time
import numpy as np
import os 

consumer_key = os.getenv("bot_consumer_key")
consumer_secret = os.getenv("bot_consumer_secret")
access_token = os.getenv("bot_access_token")
access_token_secret = os.getenv("bot_access_token_secret")


# Initialize sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#Setup Tweepy API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [2]:
#Saving real time stock data in  file
stockHistory = 'stockHistory.csv'
# Base URL for GET requests to retrieve stock data
url = "https://api.iextrading.com/1.0/stock/"
#Opening file
file=open(stockHistory,'w')

#Stock ticker for collecting data
stocks = ['AAPL', 'V', 'AMZN', 'WMT', 'NFLX', 'NKE', 'M']
# Specify the companies
target_terms = ["@Apple", "@Visa", "@amazon", "@Walmart", "@netflix", "@Nike", "@Macys"]

# Loop through (can change the range if we need to)
def getSentimentAnalysis(handler):
    # Loop through all companies
    
   # Make a dictionary to hold each type of sentiment
        company_sentiment = {
            "compound_list": [], 
            "positive_list": [], 
            "negative_list": [], 
            "neutral_list": []
        }
        public_tweets = api.search(handler, count=100, result_type="recent")

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]

            # Save all info to the corresponding lists
            company_sentiment["compound_list"].append(compound)
            company_sentiment["positive_list"].append(pos)
            company_sentiment["neutral_list"].append(neu)
            company_sentiment["negative_list"].append(neg)
        list_of_sentiment = [np.mean(company_sentiment["compound_list"]),np.mean(company_sentiment["positive_list"]),
                np.mean(company_sentiment["neutral_list"]),np.mean(company_sentiment["negative_list"])]
        return list_of_sentiment
print(str(datetime.now().time()))

10:30:33.282492


In [3]:
stock_market_open_time = time(hour=8, minute=30, second=0, microsecond=0)
stock_market_close_time = time(hour=23, minute=0, second=0, microsecond=0)

def stockPriceCollectionWithSentiment():
    counter = 0;
    stock_tweet_df = pd.DataFrame()
    stock_tweet_data = []
    while((datetime.now().time() > stock_market_open_time) & (datetime.now().time() < stock_market_close_time) & counter < 50):
        counter = counter+1
        time_var = t.mktime(datetime.now().timetuple()) * 1000
        
        record = {}
        for stock in stocks:
            print(url+stock+"/quote")
            response = req.get(url+stock+"/quote")
            record.update({"Counter":counter})
            record.update({"Corporation":response.json()['companyName']})
            record.update({"Time":time_var})
            record.update({"Stock Ticker":response.json()['symbol']})
            record.update({"Stock Price":response.json()['latestPrice']})
            
            print("Counter::"+str(counter))
            print(response.json())
            sentiment_anal = getSentimentAnalysis(target_terms[stocks.index(stock)])
            record.update({"Compound":sentiment_anal[0]})
            record.update({"Positive":sentiment_anal[1]})
            record.update({"Neutral":sentiment_anal[2]})
            record.update({"Negative":sentiment_anal[3]})
            stock_tweet_data.append(record)
        stock_tweet_df = pd.DataFrame(stock_tweet_data)
        stock_tweet_df.to_csv("Stock_Tweet_Current.csv", mode="a")
        t.sleep(60)
try:
    stockPriceCollectionWithSentiment()
except KeyboardInterrupt:
    print('\n\nKeyboard exception received. Exiting.')
    file.close()
    exit()
file.close();

https://api.iextrading.com/1.0/stock/AAPL/quote
Counter::1
{'symbol': 'AAPL', 'companyName': 'Apple Inc.', 'primaryExchange': 'Nasdaq Global Select', 'sector': 'Technology', 'calculationPrice': 'close', 'open': 169.96, 'openTime': 1512138600451, 'close': 171.05, 'closeTime': 1512162000303, 'latestPrice': 171.05, 'latestSource': 'Close', 'latestTime': 'December 1, 2017', 'latestUpdate': 1512162000303, 'latestVolume': 39731614, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 171.11, 'delayedPriceTime': 1512165595000, 'previousClose': 171.85, 'change': -0.8, 'changePercent': -0.00466, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 29860275, 'iexBidPrice': None, 'iexBidSize': None, 'iexAskPrice': None, 'iexAskSize': None, 'marketCap': 878224067600, 'peRatio': 19.44, 'week52High': 176.24, 'week52Low': 108.25, 'ytdChange': 0.47266465777012484}
https://api.iextrading.com/1.0/stock/V/quote
Counter::1
{'symbol': 'V', 'companyName': 'Vis

https://api.iextrading.com/1.0/stock/AMZN/quote
Counter::2
{'symbol': 'AMZN', 'companyName': 'Amazon.com Inc.', 'primaryExchange': 'Nasdaq Global Select', 'sector': 'Technology', 'calculationPrice': 'close', 'open': 1172.05, 'openTime': 1512138600157, 'close': 1162.35, 'closeTime': 1512162000380, 'latestPrice': 1162.35, 'latestSource': 'Close', 'latestTime': 'December 1, 2017', 'latestUpdate': 1512162000380, 'latestVolume': 4098310, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 1162.35, 'delayedPriceTime': 1512165590000, 'previousClose': 1176.75, 'change': -14.4, 'changePercent': -0.01224, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 4157847, 'iexBidPrice': None, 'iexBidSize': None, 'iexAskPrice': None, 'iexAskSize': None, 'marketCap': 560104122611, 'peRatio': 295.01, 'week52High': 1213.41, 'week52Low': 742, 'ytdChange': 0.5422532408083113}
https://api.iextrading.com/1.0/stock/WMT/quote
Counter::2
{'symbol': 'WMT', 'company

https://api.iextrading.com/1.0/stock/NFLX/quote
Counter::3
{'symbol': 'NFLX', 'companyName': 'Netflix Inc.', 'primaryExchange': 'Nasdaq Global Select', 'sector': 'Communication Services', 'calculationPrice': 'close', 'open': 186.82, 'openTime': 1512138600438, 'close': 186.82, 'closeTime': 1512162000251, 'latestPrice': 186.82, 'latestSource': 'Close', 'latestTime': 'December 1, 2017', 'latestUpdate': 1512162000251, 'latestVolume': 6214191, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 186.82, 'delayedPriceTime': 1512165519000, 'previousClose': 187.58, 'change': -0.76, 'changePercent': -0.00405, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 5721765, 'iexBidPrice': None, 'iexBidSize': None, 'iexAskPrice': None, 'iexAskSize': None, 'marketCap': 80842829707, 'peRatio': 188.71, 'week52High': 204.38, 'week52Low': 118.4, 'ytdChange': 0.46536983292807277}
https://api.iextrading.com/1.0/stock/NKE/quote
Counter::3
{'symbol': 'NKE', 'co

https://api.iextrading.com/1.0/stock/M/quote
Counter::4
{'symbol': 'M', 'companyName': "Macy's Inc", 'primaryExchange': 'New York Stock Exchange', 'sector': 'Consumer Cyclical', 'calculationPrice': 'close', 'open': 23.81, 'openTime': 1512138708633, 'close': 24.19, 'closeTime': 1512162098998, 'latestPrice': 24.19, 'latestSource': 'Close', 'latestTime': 'December 1, 2017', 'latestUpdate': 1512162098998, 'latestVolume': 14772735, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 24.22, 'delayedPriceTime': 1512165434275, 'previousClose': 23.8, 'change': 0.39, 'changePercent': 0.01639, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 17053960, 'iexBidPrice': None, 'iexBidSize': None, 'iexAskPrice': None, 'iexAskSize': None, 'marketCap': 7367281363, 'peRatio': 7.56, 'week52High': 43.79, 'week52Low': 17.405, 'ytdChange': -0.31336928753902915}


Keyboard exception received. Exiting.
